In [14]:
import datetime as datetime
import time as time

import pandas as pd
import talib
from backtesting import Strategy, Backtest
from backtesting.lib import crossover, resample_apply

In [15]:
#IMPORT LIST OF SP500 STOCKS HERE IN THE TICKER VARIABLE TO RUN TESTS ON ALL OF THE INDIVIDUAL
#STOCKS

#Ticker in url
ticker = 'AAPL'

#Timeperiods of data set "Y/M/D/time"
#this will help when selecting the desired dates and will pull the data set from yahoo finance.
period1 = int(time.mktime(datetime.datetime(2013, 1, 1, 23, 59).timetuple()))
period2 = int(time.mktime(datetime.datetime(2023, 1, 12, 23, 59).timetuple()))
interval = '1d' # 1wk, 1m

#Yahoo Finance url
url = f'https://query1.finance.yahoo.com/v7/finance/download/{ticker}?period1={period1}&period2={period2}&interval={interval}&events=history&includeAdjustedClose=true'

#defining df to backtesting requirements
df = pd.read_csv(url)
columns = ['Date', 'Open', 'High', 'Low', 'Close', 'adj close', 'Volume']
df.columns = columns

#index df to date time index to fit backtesting.py
#df requirements
df = df.set_index(pd.DatetimeIndex(df['Date'].values))

#dropping columns that are not necessary for backtesting.py format
df.drop('Date', inplace=True, axis=1)
df.drop('adj close', inplace=True, axis=1)
df

,Open,High,Low,Close,Volume
2013-01-02,19.779285,19.821428,19.343929,19.608213,560518000
2013-01-03,19.567142,19.631071,19.321428,19.360714,352965200
2013-01-04,19.177500,19.236786,18.779642,18.821428,594333600
2013-01-07,18.642857,18.903570,18.400000,18.710714,484156400
2013-01-08,18.900356,18.996071,18.616072,18.761070,458707200
...,...,...,...,...,...
2023-01-06,126.010002,130.289993,124.889999,129.619995,87686600
2023-01-09,130.470001,133.410004,129.889999,130.149994,70790800
2023-01-10,130.259995,131.259995,128.119995,130.729996,63896200
2023-01-11,131.250000,133.509995,130.460007,133.490005,69458900


In [16]:
#the overall trading strategy function
class MACD(Strategy):

    #defines the premade trading parameters imported from talib
    #(trading parameter, data column being used, trading window)
    def init(self):

        #MACD variables
        self.macd, self.macdsignal, self.macdhist = self.I(talib.MACD, self.data.Close, fastperiod=12, slowperiod=26, signalperiod=9)
        #A comparison from the TA-lib directory
        #macd, macdsignal, macdhist = MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)

    def next(self):

        if crossover(self.macdsignal, self.macd):
            #if this statement is true the below command signals a sell.
            self.position.close()
            # self.sell()

        elif crossover(self.macd, self.macdsignal):
            #buy command
            # self.position.close()
            self.buy()


#bt variable runs the backtest dependant on the data, strategy, and cash
#other parameters can be added to more complex strategies. Refer to
#backtesting.py on github
bt = Backtest(df, MACD, cash = 10_000)
stats = bt.run()
stats
#the plotting function does not work in python 3.8.7 so it needs to be
#run in a earlier python like python 3.6 to graph the trades
#bt.plot()

Start                     2013-01-02 00:00:00
End                       2023-01-12 00:00:00
Duration                   3662 days 00:00:00
Exposure Time [%]                   52.573238
Equity Final [$]                 71986.290283
Equity Peak [$]                   83812.44053
Return [%]                         619.862903
Buy & Hold Return [%]              580.378186
Return (Ann.) [%]                   21.764687
Volatility (Ann.) [%]               22.081149
Sharpe Ratio                         0.985668
Sortino Ratio                        1.843779
Calmar Ratio                         0.809566
Max. Drawdown [%]                  -26.884401
Avg. Drawdown [%]                    -3.30305
Max. Drawdown Duration      521 days 00:00:00
Avg. Drawdown Duration       38 days 00:00:00
# Trades                                   82
Win Rate [%]                        56.097561
Best Trade [%]                      26.334044
Worst Trade [%]                    -10.692964
Avg. Trade [%]                    

In [17]:
bt.plot()

Row(id='3208', ...)

In [18]:
stats.tail()
stats['_trades']

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,665,80,94,15.016071,15.680357,441.750190,0.044238,2013-04-29,2013-05-17,18 days
1,646,103,108,16.160713,15.589286,-369.141842,-0.035359,2013-05-31,2013-06-07,7 days
2,670,127,165,15.013929,17.357143,1569.953380,0.156069,2013-07-05,2013-08-28,54 days
3,673,186,214,17.277857,18.719643,970.321978,0.083447,2013-09-27,2013-11-06,40 days
4,642,230,241,19.624287,19.822144,127.024194,0.010082,2013-11-29,2013-12-16,17 days
...,...,...,...,...,...,...,...,...,...,...
77,492,2369,2379,149.070007,133.130005,-7842.480984,-0.106930,2022-05-31,2022-06-14,14 days
78,468,2386,2427,139.899994,167.080002,12720.243744,0.194282,2022-06-24,2022-08-23,60 days
79,538,2466,2480,145.490005,137.110001,-4508.442152,-0.057598,2022-10-18,2022-11-07,20 days
80,495,2485,2496,148.970001,141.399994,-3747.153465,-0.050816,2022-11-14,2022-11-30,16 days


In [19]:
#the overall trading strategy function
class Stochastic(Strategy):

    #defines the premade trading parameters imported from talib
    #(trading parameter, data column being used, trading window)
    def __init__(self, broker, data, params):
        super().__init__(broker, data, params)
        self.stochd = None
        self.stochk = None

    def init(self):
        #STOCH variables
        self.stochk, self.stochd = self.I(talib.STOCH, self.data.High, self.data.Low, self.data.Close, fastk_period=14, slowk_period=3, slowd_period=3)

        #slowk, slowd = STOCH(high, low, close, fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
    def next(self):

        if self.stochk > 80 and self.stochd > 80 and crossover(self.stochk, self.stochd):
            #if this statement is true the below command signals a sell.
            self.position.close()
        elif self.stochk < 20 and self.stochd < 20 and crossover(self.stochd, self.stochk):
            #buy command
            self.buy()

#bt variable runs the backtest dependant on the data, strategy, and cash
#other parameters can be added to more complex strategies. Refer to
#backtesting.py on github
bt = Backtest(df, Stochastic)
stats = bt.run()
stats


Start                     2013-01-02 00:00:00
End                       2023-01-12 00:00:00
Duration                   3662 days 00:00:00
Exposure Time [%]                   30.205859
Equity Final [$]                 22626.199719
Equity Peak [$]                  28935.429574
Return [%]                         126.261997
Buy & Hold Return [%]              580.378186
Return (Ann.) [%]                    8.486808
Volatility (Ann.) [%]               19.210787
Sharpe Ratio                         0.441773
Sortino Ratio                        0.716697
Calmar Ratio                         0.314568
Max. Drawdown [%]                  -26.979282
Avg. Drawdown [%]                   -4.620624
Max. Drawdown Duration      680 days 00:00:00
Avg. Drawdown Duration       48 days 00:00:00
# Trades                                   15
Win Rate [%]                             80.0
Best Trade [%]                      24.733863
Worst Trade [%]                    -13.424725
Avg. Trade [%]                    

In [20]:
stats['_trades']

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,654,41,56,15.278571,16.596071,861.645000,0.086232,2013-03-04,2013-03-25,21 days
1,1,122,164,14.258929,17.785713,3.526784,0.247339,2013-06-27,2013-08-27,61 days
2,704,116,164,15.407143,17.785713,1674.513280,0.154381,2013-06-19,2013-08-27,69 days
3,650,258,337,19.279642,21.492857,1438.589750,0.114795,2014-01-10,2014-05-06,116 days
4,513,747,890,27.227501,24.730000,-1281.218013,-0.091727,2015-12-18,2016-07-15,210 days
5,467,969,986,27.132500,27.900000,358.422500,0.028287,2016-11-04,2016-11-30,26 days
6,328,1282,1296,39.775002,44.087502,1414.500000,0.108422,2018-02-05,2018-02-26,21 days
7,347,1320,1393,41.660000,47.880001,2158.340347,0.149304,2018-04-02,2018-07-16,105 days
8,370,1484,1626,44.932499,49.919998,1845.374630,0.111000,2018-11-21,2019-06-19,210 days
9,298,1815,1849,61.847500,75.805000,4159.335000,0.225676,2020-03-19,2020-05-07,49 days


## MACD and Stochastic: A Double-Cross Strategy

https://www.investopedia.com/articles/trading/08/macd-stochastic-double-cross.asp#:~:text=The%20Strategy,-First%2C%20look%20for&text=When%20applying%20the%20stochastic%20and,days%20of%20placing%20your%20trade.

### The Strategy

First, look for the bullish crossovers to occur within two days of each other. When applying the stochastic and MACD double-cross strategy, ideally, the crossover occurs below the 50-line on the stochastic to catch a longer price move. And preferably, you want the histogram value to already be or move higher than zero within two days of placing your trade.

Also note the MACD must cross slightly after the stochastic, as the alternative could create a false indication of the price trend or place you in a sideways trend.

Finally, it is safer to trade stocks trading above their 200-day moving averages, but it is not an absolute necessity.



In [21]:
#the overall trading strategy function
class DoubleCross(Strategy):

    #defines the premade trading parameters imported from talib
    #(trading parameter, data column being used, trading window)
    def __init__(self, broker, data, params):
        super().__init__(broker, data, params)
        self.stochd = None
        self.stochk = None
        self.macdhist = None
        self.macdsignal = None
        self.macd = None
        self.db_lookback = -4

    def init(self):
        #STOCH variables
        self.macd, self.macdsignal, self.macdhist =  self.I(talib.MACD, self.data.Close, fastperiod=12, slowperiod=26, signalperiod=9)
        self.stochk, self.stochd = resample_apply("D", talib.STOCH, self.data.High, self.data.Low, self.data.Close, fastk_period=14, slowk_period=3, slowd_period=3)

    def next(self):
        if crossover(self.macdsignal, self.macd):
            #if this statement is true the below command signals a sell.
            for i in range(-1, self.db_lookback, -1):
                if 80 < self.stochk[i] < self.stochd[i]:
                    self.position.close()
                    break
        elif crossover(self.macd, self.macdsignal):
            for i in range(-1, self.db_lookback, -1):
                if 50 > self.stochk[i] > self.stochd[i]:
                    self.buy()
                    break


bt = Backtest(df, DoubleCross, cash = 10_000, exclusive_orders=True)
stats = bt.run()
stats
#the plotting function does not work in python 3.8.7 so it needs to be
#run in a earlier python like python 3.6 to graph the trades
#bt.plot()

Start                     2013-01-02 00:00:00
End                       2023-01-12 00:00:00
Duration                   3662 days 00:00:00
Exposure Time [%]                   50.118765
Equity Final [$]                 70671.033067
Equity Peak [$]                  82191.253067
Return [%]                         606.710331
Buy & Hold Return [%]              580.378186
Return (Ann.) [%]                   21.540893
Volatility (Ann.) [%]               23.922954
Sharpe Ratio                         0.900428
Sortino Ratio                        1.684516
Calmar Ratio                         0.927257
Max. Drawdown [%]                  -23.230762
Avg. Drawdown [%]                   -3.051614
Max. Drawdown Duration      520 days 00:00:00
Avg. Drawdown Duration       38 days 00:00:00
# Trades                                   49
Win Rate [%]                        67.346939
Best Trade [%]                      27.738223
Worst Trade [%]                     -12.55958
Avg. Trade [%]                    

In [22]:
bt.plot()

Row(id='4141', ...)

In [23]:
stats.tail()
stats['_trades']

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,665,80,94,15.016071,15.680357,441.750190,0.044238,2013-04-29,2013-05-17,18 days
1,695,127,165,15.013929,17.357143,1628.533730,0.156069,2013-07-05,2013-08-28,54 days
2,698,186,230,17.277857,19.624287,1637.808140,0.135806,2013-09-27,2013-11-29,63 days
3,698,230,241,19.624287,19.822144,138.104186,0.010082,2013-11-29,2013-12-16,17 days
4,730,279,328,18.950357,18.895000,-40.410610,-0.002921,2014-02-11,2014-04-23,71 days
5,730,328,330,18.895000,20.161785,924.753050,0.067043,2014-04-23,2014-04-25,2 days
6,730,330,345,20.161785,21.022499,628.321220,0.042690,2014-04-25,2014-05-16,21 days
7,623,409,421,24.622499,24.712500,56.070623,0.003655,2014-08-18,2014-09-04,17 days
8,599,455,484,25.709999,28.937500,1933.273099,0.125535,2014-10-22,2014-12-03,42 days
9,617,518,544,28.075001,32.240002,2569.805617,0.148353,2015-01-23,2015-03-03,39 days


In [ ]:

#the plotting function does not work in python 3.8.7 so it needs to be
#run in a earlier python like python 3.6 to graph the trades
bt.plot()

In [ ]:
stats = bt.optimize(db_lookback=range(-1, -10, -1),
                    maximize='Equity Final [$]')
stats['_strategy']